In [2]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
import os
import pickle

SCOPES = [
    "https://www.googleapis.com/auth/forms.body",
    "https://www.googleapis.com/auth/forms.responses.readonly",
    "https://www.googleapis.com/auth/drive.file"
]

creds = None

if os.path.exists("token.pickle"):
    with open("token.pickle", "rb") as token:
        creds = pickle.load(token)

if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "credentials.json", SCOPES
        )
        creds = flow.run_local_server(port=0)

    with open("token.pickle", "wb") as token:
        pickle.dump(creds, token)

forms_service = build("forms", "v1", credentials=creds)

In [5]:
form_body = {
    "info": {
        "title": "Customer Feedback Form",
        "documentTitle": "Customer Feedback Form"
    }
}

form = forms_service.forms().create(body=form_body).execute()

form_id = form["formId"]
print("Form created!")
print("Form ID:", form_id)
print("Responder URL:", form["responderUri"])

Form created!
Form ID: 1mDmmNgJ6K9CEq-kLjpoPjFlYrtlNjaWL7tYaTYH4Lgg
Responder URL: https://docs.google.com/forms/d/e/1FAIpQLSfAz-qHHv_c5WR9jC1CtzwCBWlfgOXGYl2THKd8xVsUOfRpVQ/viewform


In [6]:
requests = [
    {
        "createItem": {
            "item": {
                "title": "How satisfied are you with our service?",
                "questionItem": {
                    "question": {
                        "required": True,
                        "choiceQuestion": {
                            "type": "RADIO",
                            "options": [
                                {"value": "Very satisfied"},
                                {"value": "Satisfied"},
                                {"value": "Neutral"},
                                {"value": "Dissatisfied"},
                                {"value": "Very dissatisfied"}
                            ],
                            "shuffle": False
                        }
                    }
                }
            },
            "location": {"index": 0}
        }
    },
    {
        "createItem": {
            "item": {
                "title": "Additional comments",
                "questionItem": {
                    "question": {
                        "required": False,
                        "textQuestion": {
                            "paragraph": True
                        }
                    }
                }
            },
            "location": {"index": 1}
        }
    }
]

batch_update_body = {"requests": requests}

forms_service.forms().batchUpdate(
    formId=form_id,
    body=batch_update_body
).execute()

print("Questions added!")

Questions added!


In [4]:
form = forms_service.forms().get(formId=form_id).execute()

print("Form title:", form["info"]["title"])
for item in form.get("items", []):
    print("-", item["title"])

Form title: Customer Feedback Form
- How satisfied are you with our service?
- Additional comments
